In [2]:
from IPython.core.display import display, HTML
display(HTML('<style>.container {width:100% !important;}</style>'))


In [ ]:
import csv
import pandas as pd
from daomop.storage import tap_query
import os
import time 
import subprocess
from astropy import wcs
from astropy.io import fits
from astropy.nddata.utils import Cutout2D
import numpy as np
import random
def get_filenames(ra, dec,i): #0.001
    CUTOUT_PATH ="/home/robbie/repos/HSTLens/CC_batch_processing/batchjob_3/images/"+str(ra)+"_"+str(dec)+".fits"
    OUT_PATH= "/home/robbie/repos/HSTLens/CC_batch_processing/batchjob_3/out/"+str(ra)+"_"+str(dec)+".fits"

    
    query = '''SELECT caom2.Plane.publisherID, caom2.Plane.planeURI
    FROM caom2.observation 
    JOIN caom2.plane ON caom2.observation.obsID = caom2.plane.obsID 
    WHERE CONTAINS( CIRCLE('ICRS',''' + str(ra) + ',' +str(dec) + ''',0.001), caom2.plane.position_bounds)=1 
    AND caom2.observation.collection = 'HST' 
    AND caom2.observation.instrument_name = 'ACS/WFC' 
    AND caom2.Plane.energy_bandpassName ='F814W'
    ''' #'F814W' F606W,F850W F475WAND   AND 
    print ra
    out_query = tap_query(query)

    '''store the pubID and filenames'''
    success=False
    for entry in range(len(out_query["publisherID"])):#
#         if (True):
        if (("0-"  in out_query["publisherID"][entry]) and ("PRODUCT"  in out_query["publisherID"][entry])) :
            success=True
            start = out_query["publisherID"][entry].find('HST?') + 4 #add 4 to get to the and of the HST?
            end = out_query["publisherID"][entry].find('/j', start)
            print(out_query["publisherID"][entry][start:end])
            try:

                subprocess.check_output(["wget", "-O", CUTOUT_PATH,
                                 "http://www.cadc-ccda.hia-iha.nrc-cnrc.gc.ca/caom2ops/cutout?uri=mast:HST/product/"+
                                 str(out_query["publisherID"][entry][start:end])+"_drc.fits&cutout=Circle+ICRS+" + str(ra) + "+" +
                                 str(dec) + "+0.002"])#0.002
                break
            except:
                print('fail')
                
    if success==False: 
        print('couldnt find filename')
        bad_objects.append(i)
        return
        
    data, hdr = fits.getdata(CUTOUT_PATH, 1, header=True) #open the dwnlded image
    w = wcs.WCS(hdr) 
    pixcrd = w.wcs_world2pix([[float(ra), float(dec)]], 0) #find the pixel crds to center on
    centered_cut = Cutout2D(data, (pixcrd[0][0], pixcrd[0][1]), (100, 100)) #make cutout
    fits.writeto(OUT_PATH, centered_cut.data, header=hdr, overwrite =True) #save cutout
        
crd_dict = {'ra':[],'dec':[]}
pos=0 #to alternate btw ra and dec

my_file = open("unformatted_masterlens","r") #open and read the raw masterlens data
lines = my_file.readlines()
my_file.close()

for line in lines:

  
    start = line.find('[') 
    if start != -1: #if it found somthing
        start+=1
        end = line.find(']') -1

        if line[end-1]=='\xc2':
            word = line[start:end-1] 

            if pos==0:
                crd_dict['ra'].append(word)
                pos=1
            else:
                crd_dict['dec'].append(word)
                pos=0
                
crd_df = pd.DataFrame(crd_dict)
crd_df.to_csv('masterlens.csv')
bad_objects = []

for i in range(len(crd_dict['ra'])):
    try:
        get_filenames(crd_dict['ra'][i], crd_dict['dec'][i], i)
    except:
        bad_objects.append(i)
        
with open("testcsv", 'wb') as myfile: #save the list of bad images
    wr = csv.writer(myfile)
    wr.writerow(c)


-30.352722
couldnt find filename
-30.382194
couldnt find filename
-10.15308799
couldnt find filename
+16.4378587777
couldnt find filename
-0.06896
couldnt find filename
5.67049154
6.3479305559
j8qu01020
